In [ ]:
import awswrangler as wr
from datetime import datetime, timedelta
now = datetime.now() - timedelta(hours = 4)

# episodes = ["1716761117540","test"] # pageids of episodes to take
# programs = ["1460018486818"] # program pageids of programs to take

query = f"""
SELECT
vrtmax_catalog.mediacontent_media_id as mediacontent_media_id,
mediacontent_pageid,
mediacontent_page_description,
mediacontent_pagetitle_program,
subtitle
FROM (
SELECT mediacontent_media_id,
mediacontent_pageid,
mediacontent_page_description,
mediacontent_pagetitle_program
FROM 
derived_prod.vrtmax_catalog_mediaid_history WHERE year = {now.year} and month = {now.month} and day = {now.day} and hour = {now.hour} 
AND offering_publication_planneduntil > CURRENT_TIMESTAMP
) vrtmax_catalog
LEFT JOIN (
SELECT 
mediacontent_media_id,
array_join(array_agg(subtitle ORDER BY starttc ASC), ' ') as subtitle
FROM (
SELECT 
MIN(subtitle) as subtitle,
mediacontent_media_id,
starttc
FROM (
SELECT 
subtitle,
mediacontent_media_id,
starttc
FROM
(
SELECT 
objid, subtitle
FROM (
SELECT objid, subtitle, 
row_number() OVER (partition by subtitle ORDER BY objid DESC) as rn 
FROM (
SELECT objid, a_vrt_subti2  as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_s1"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid, a_vrt_subti3  as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_s2"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid, a_vrt_subtit as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_s3"   -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid, a_vrt_subti4 as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_s4"   -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION 
SELECT objid, a_vrt_speech as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_sp"   -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION 
SELECT objid, a_vrt_subti1 as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_su"   -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
)
)
WHERE rn = 1
) a
LEFT JOIN (
SELECT objid, segmentcontentid, starttc
FROM (
SELECT objid, segmentcontentid, starttc,
row_number() OVER (partition by segmentcontentid ORDER BY objid DESC) as rn 
FROM (
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_sub1"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_sub2"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_sub3"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_sub4"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_subt"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
)
GROUP BY objid, segmentcontentid, starttc
) 
WHERE rn = 1
) b
ON a.objid = b.segmentcontentid
LEFT JOIN (
SELECT objid,  a_vrt_progra ,a_vrt_media_
FROM (
SELECT objid,  a_vrt_progra ,a_vrt_media_,
row_number() OVER (partition by objid ORDER BY a_vrt_media_ DESC) as rn 
FROM "operationaldata_nonsensitive_prod"."depot_dmo_program"  WHERE import_year = 2024 and import_month = 6 and import_day = 9
) WHERE rn = 1
) c
ON b.objid = c.objid
INNER JOIN (
SELECT mediacontent_media_id, 	mediacontent_media_origin_id  
FROM (
SELECT mediacontent_media_id, mediacontent_media_origin_id, row_number() OVER (PARTITION BY mediacontent_media_origin_id   ORDER BY 	contactmoment_ingest_timestamp DESC)  as rn FROM "marketing_prod"."dtr_vd_dms_audio_video_metadata_datariver"
)
WHERE rn = 1
) d
ON c.a_vrt_media_ = d.mediacontent_media_origin_id
)
GROUP BY mediacontent_media_id, starttc
)
GROUP BY mediacontent_media_id
) subtitles  
ON vrtmax_catalog.mediacontent_media_id = subtitles.mediacontent_media_id
"""

df = wr.athena.read_sql_query(sql=query, database="derived_prod")